# グローバーのアルゴリズム
グローバーのアルゴリズムは未整序のデータの検索が高速にできるというアルゴリズムです。マーキングの部分にあたるオラクルと呼ばれる関数部分に検索のアルゴリズムを入れましょう。

全体の流れは下記の通りです。

１、重ね合わせ
２、探したいデータのマーキング
３、マーキングを可視化する振幅増幅反転

マーキングと振幅増幅反転を適切な回数繰り返すと精度が上がります。こちらをBlueqatで実装してみたいと思います。

## マーキング
まず2量子ビットのGroverからです。2量子ビットの組み合わせの場合の数は、00,01,10,11の4通りです。その中から特定の組み合わせを抜き出してみたいと思います。それを実現するにはゲート操作をつかって、「解に対応する状態ベクトルだけに-1がかかる対角行列を数学的に作り」ます。
 

In [0]:
'''
#00
[[-1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]]

#01
[[ 1  0  0  0]
 [ 0 -1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]]

#10
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0 -1  0]
 [ 0  0  0  1]]

#11
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0 -1]]
 '''

このように、マーキングしたい状態ベクトルに対応する部分に-1を設定した行列を作ってかけます。

## マーキング
マーキング回路は下記のように重ね合わせの後に上記のゲートをかけます。下記のようにHのあとに各々のオラクルをつけます。*はコントロールゲート、Zはターゲットゲートです。対応するblueqatコードも併記します。

In [0]:
'''
#00
--H--S--*--S--
--H--S--Z--S--
'''

Circuit(2).h[:].s[:].cz[0,1].s[:]

'''
#01
--H-----*-----
--H--S--Z--S--
'''

Circuit(2).h[:].s[1].cz[0,1].s[1]

'''
#10
--H--S--*--S--
--H-----Z-----
'''

Circuit(2).h[:].s[0].cz[0,1].s[0]

'''
#11
--H-----*-----
--H-----Z-----
'''

Circuit(2).h[:].cz[0,1]

## 振幅増幅反転
振幅増幅反転は、対角項が-1、非対角項が+1の行列を用意することでマーキングした回路にかけることで、マーキングしたものの振幅が増幅されます。振幅増幅反転のユニタリ変換は各パターン共通となっています。こちらをBlueqatに直してみます。

In [0]:
Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].m[:]

'''
--H-X-*-X-H--
--H-X-Z-X-H--
'''


## 回路の実装
では、実際の回路の実装です。

In [8]:
from blueqat import Circuit

#振幅増幅反転
a = Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].m[:]

'''
#00回路
--H--S--*--S----H-X-*-X-H--
--H--S--Z--S----H-X-Z-X-H--
'''

(Circuit(2).h[:].s[:].cz[0,1].s[:] + a).run(shots=100)

Counter({'00': 100})

In [9]:
'''
#01回路
--H-----*-------H-X-*-X-H--
--H--S--Z--S---H-X-Z-X-H--
'''

(Circuit(2).h[:].s[1].cz[0,1].s[1] + a).run(shots=100)

Counter({'01': 100})

In [10]:
'''
#10回路
--H--S--*--S----H-X-*-X-H--
--H-----Z--------H-X-Z-X-H--
'''
(Circuit(2).h[:].s[0].cz[0,1].s[0] + a).run(shots=100)

Counter({'10': 100})

In [11]:
'''
#11回路
--H-----*-------H-X-*-X-H--
--H-----Z-------H-X-Z-X-H--
'''
(Circuit(2).h[:].cz[0,1] + a).run(shots=100)

Counter({'11': 100})

このようにできました。Groverのアルゴリズムが身近になりました。